In [1]:
import pandas as pd
from config_GAM2025 import gam_info
import functions

In [2]:
# Load country mapping
country_map = pd.read_excel(f"../../{gam_info['lookup_file']}", sheet_name='CountryID')[['PlaceID', 'YouTube Codes']]
# Load country mapping
week_map = pd.read_excel(f"../../{gam_info['lookup_file']}", sheet_name='GAM Period')[['w/c', 'WeekNumber_finYear']]


In [3]:
# Utility functions
def load_excel(path):
    return pd.read_excel(path, engine='openpyxl')

def load_csv(path):
    return pd.read_csv(path)

def standardize_country_codes(df, column='Country Code'):
    return df.replace({column: {'WLF': 'WFI', '* Total': 'Total'}})

def run_comparison(original_df, new_df, column_mapping, key_columns, method='integer', threshold=0.0001):
    if method == 'integer':
        return compare_dataframes_integer(original_df, new_df, column_mapping, key_columns)
    elif method == 'percentage':
        return compare_dataframes_percentage(original_df, new_df, column_mapping, key_columns, threshold)
    else:
        raise ValueError("Unknown comparison method")

In [4]:
def compare_dataframes_integer(original_df, new_df, column_mapping, key_columns_new):
    """
    Compare two DataFrames and return rows that are missing or different.

    Parameters:
    - original_df: DataFrame from the original source
    - new_df: DataFrame from the new source
    - column_mapping: dict mapping original_df column names to new_df column names
    - key_columns_new: list of key columns using new_df naming

    Returns:
    - missing_from_new: rows in original_df not found in new_df
    - differing_rows: rows where key matches but mapped columns differ
    """

    # Rename original_df to match new_df column names
    original_df_renamed = original_df.rename(columns=column_mapping)

    # Ensure all required columns exist
    all_columns = list(column_mapping.values())
    original_subset = original_df_renamed[all_columns].copy()
    new_subset = new_df[all_columns].copy()

    # Round numeric columns to nearest integer
    for col in all_columns:
        if pd.api.types.is_numeric_dtype(original_subset[col]) and pd.api.types.is_numeric_dtype(new_subset[col]):
            original_subset[col] = original_subset[col].round(0).astype('Int64')
            new_subset[col] = new_subset[col].round(0).astype('Int64')
        
    # Merge to find differences
    merged = pd.merge(
        original_subset,
        new_subset,
        on=key_columns_new,
        how='outer',
        suffixes=('_orig', '_new'),
        indicator=True
    )

    # Missing rows: present in original but not in new
    missing_from_new = merged[merged['_merge'] == 'left_only']

    # Differing rows: same keys but different values
    comparison_cols = [col for col in all_columns if col not in key_columns_new]
        
    differing_rows = merged[
        (merged['_merge'] == 'both') &
        merged[[f"{col}_orig" for col in comparison_cols]].ne(
            merged[[f"{col}_new" for col in comparison_cols]].values
        ).any(axis=1)
    ]

    return missing_from_new, differing_rows

In [5]:
def compare_dataframes_percentage(original_df, new_df, column_mapping, key_columns_new, threshold=0.0001):
    """
    Compare two DataFrames and return rows that are missing or have percentage differences.

    Parameters:
    - original_df: DataFrame from the original source
    - new_df: DataFrame from the new source
    - column_mapping: dict mapping original_df column names to new_df column names
    - key_columns_new: list of key columns using new_df naming
    - threshold: minimum absolute difference to consider as significant

    Returns:
    - missing_from_new: rows in original_df not found in new_df
    - differing_rows: rows where key matches but mapped columns differ beyond threshold
    """

    # Rename original_df to match new_df column names
    original_df_renamed = original_df.rename(columns=column_mapping)

    # Ensure all required columns exist
    all_columns = list(column_mapping.values())
    original_subset = original_df_renamed[all_columns].copy()
    new_subset = new_df[all_columns].copy()

    # Merge to find differences
    merged = pd.merge(
        original_subset,
        new_subset,
        on=key_columns_new,
        how='outer',
        suffixes=('_orig', '_new'),
        indicator=True
    )

    # Missing rows: present in original but not in new
    missing_from_new = merged[merged['_merge'] == 'left_only']

    # Compute differences
    comparison_cols = [col for col in all_columns if col not in key_columns_new]
    for col in comparison_cols:
        merged[f"{col}_diff"] = merged[f"{col}_new"] - merged[f"{col}_orig"]

    # Filter rows where any difference exceeds threshold
    diff_mask = merged['_merge'] == 'both'
    for col in comparison_cols:
        diff_mask &= merged[f"{col}_diff"].abs() > threshold

    differing_rows = merged[diff_mask]

    return missing_from_new, differing_rows


In [13]:
# Dataset configuration
datasets = [
    {
        "name": "WSC ",
        "original_path": "../data/interim/mk_wsc_social_weekly_data.csv",
        "new_path": f"../data/combinePlatforms/{gam_info['file_timeinfo']}_weekly_WSC.csv",
        "column_mapping": {
            "Service Code": "ServiceID",
            "Country Code": "PlaceID",
            "Platform Code": "PlatformID",
            "w/c": "w/c",
            "YearGAE": "YearGAE",
            "Reach": "Reach"
        },
        "key_columns": ["ServiceID", "PlaceID", "PlatformID", "w/c", "YearGAE"],
        "method": "integer",
        "preprocess": {
            "standardize_country": False,
            "week_mapping": True,
            "farsi": True
        }
    },
    
]

In [14]:

# Execute comparisons
for ds in datasets:
    # TODO - test currently doesn't catch additional things in my dataset that are not in minnie's 
    # e.g. I included Studios for UK / Youtube and Minnie did not - that did not show up here
    print(f"\n--- Processing {ds['name']} ---")

    orig = load_excel(ds["original_path"]) if ds["original_path"].endswith(".xlsx") else load_csv(ds["original_path"])
    new  = load_excel(ds["new_path"]) if ds["new_path"].endswith(".xlsx") else load_csv(ds["new_path"])

    # Special preprocessing for Country Percentage dataset
    if ds["name"] == "Country Percentage":
        
        # Rename 'Country' to 'YouTube Codes' in original data and merge with mapping
        orig = orig.rename(columns={'Country': 'YouTube Codes'})
        orig = orig.merge(country_map, on='YouTube Codes', how='left').drop(columns=['YouTube Codes'])

    if "Country Code" in orig.columns:
        orig = standardize_country_codes(orig)
    if "Country Code" in new.columns:
        new = standardize_country_codes(new)

    # Rename columns according to mapping
    orig = orig.rename(columns={k: v for k, v in ds["column_mapping"].items() if k in orig.columns})
    new  = new.rename(columns={k: v for k, v in ds["column_mapping"].items() if k in new.columns})

    if ds['preprocess']['farsi']:
        orig['ServiceID'] = orig['ServiceID'].replace("FAR", "PER")
    # Special preprocessing for Country Percentage dataset
    if ds['preprocess']['week_mapping']:
        # add w/c using Week Number
        orig = orig.merge(week_map, left_on='Week Number', right_on='WeekNumber_finYear',
                                              how='left').drop(columns=['Week Number', 'WeekNumber_finYear'])

    '''# Special preprocessing for Country Percentage dataset
    if ds["name"] in ["GNL Weekly", "WSL Weekly", "WOR Weekly", 
                      "WSE Weekly", "MA- Weekly", "FOA Weekly", 
                      "AXE Weekly", "AX2 Weekly", "ANW Weekly",
                      "ANY Weekly", "TOT Weekly", "ALL Weekly",
                     ]:
        
        # Rename 'Country' to 'YouTube Codes' in original data and merge with mapping
        orig = orig.merge(week_map, left_on='Week Number', right_on='WeekNumber_finYear',
                                              how='left').drop(columns=['Week Number', 'WeekNumber_finYear'])
    '''
    # Ensure 'w/c' columns are datetime in both DataFrames
    if 'w/c' in orig.columns:
        orig['w/c'] = pd.to_datetime(orig['w/c'], errors='coerce')
    if 'w/c' in new.columns:
        new['w/c'] = pd.to_datetime(new['w/c'], errors='coerce')

    missing, different = run_comparison(
        orig, new,
        ds["column_mapping"],
        ds["key_columns"],
        method=ds.get("method", "integer"),
        threshold=ds.get("threshold", 0.0001)
    )

    print("Rows missing from new:")
    display(missing)
    print("Rows with differences:")
    if len(different) > 0:
        different['diff'] = different['Reach_orig'] - different['Reach_new']
        display(different.sort_values('diff', ascending=False))
    else:
        display(different)


--- Processing WSC  ---


/var/folders/gz/pq5c3fbj5rs1tz_5w1hycq4h0000gn/T/ipykernel_5936/1482102022.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path)


Rows missing from new:


,ServiceID,PlaceID,PlatformID,w/c,YearGAE,Reach_orig,Reach_new,_merge
0,AFA,AFG,WSC,2024-04-01,2025,129,<NA>,left_only
1,AFA,AFG,WSC,2024-04-08,2025,147,<NA>,left_only
2,AFA,AFG,WSC,2024-04-15,2025,152,<NA>,left_only
3,AFA,AFG,WSC,2024-04-22,2025,150,<NA>,left_only
4,AFA,AFG,WSC,2024-04-29,2025,156,<NA>,left_only
...,...,...,...,...,...,...,...,...
572179,YOR,ZAM,WSC,2025-02-24,2025,264,<NA>,left_only
572180,YOR,ZAM,WSC,2025-03-03,2025,222,<NA>,left_only
572181,YOR,ZAM,WSC,2025-03-10,2025,272,<NA>,left_only
572182,YOR,ZAM,WSC,2025-03-17,2025,255,<NA>,left_only


Rows with differences:


,ServiceID,PlaceID,PlatformID,w/c,YearGAE,Reach_orig,Reach_new,_merge,diff
476097,TOT,IND,WSC,2025-03-17,2025,40423953,26191487,both,14232466
47843,ANY,IND,WSC,2025-03-17,2025,40420902,26189661,both,14231241
476098,TOT,IND,WSC,2025-03-24,2025,33728690,19584199,both,14144491
47844,ANY,IND,WSC,2025-03-24,2025,33726220,19581934,both,14144286
7532,ALL,IND,WSC,2025-03-24,2025,34632330,21264382,both,13367948
...,...,...,...,...,...,...,...,...,...
14535,ALL,USA,WSC,2024-10-07,2025,14799586,26839262,both,-12039676
483096,TOT,USA,WSC,2024-11-11,2025,4839695,20253708,both,-15414013
54842,ANY,USA,WSC,2024-11-11,2025,4838719,20253335,both,-15414616
256409,GNL,USA,WSC,2024-11-11,2025,2538402,18398900,both,-15860498
